In [1]:
import logging; logging.basicConfig(level=logging.DEBUG)

In [38]:
import mpm
import mpm.api
import mpm.ui.gtk
reload(mpm)
reload(mpm.api)
reload(mpm.ui.gtk)

<module 'mpm.ui.gtk' from 'C:\Users\chris\Documents\GitHub\mpm\mpm\ui\gtk\__init__.pyc'>

In [36]:
import conda_helpers as ch
reload(ch)

<module 'conda_helpers' from 'conda_helpers\__init__.py'>

In [7]:
import yaml

In [11]:
print yaml.dump({'channels': ['sci-bots', 'microdrop-plugins', 'wheeler-microfluidics', 'conda-forge', 'defaults']}, default_flow_style=False)

channels:
- sci-bots
- microdrop-plugins
- wheeler-microfluidics
- conda-forge
- defaults



In [41]:
# mpm.ui.gtk.update_plugin_dialog(package_name=['foobar', 'microdrop.mr-box-plugin'],
#                                 update_args=['--dry-run'])
# print ch.conda_exec('search', '--json', '-c', 'microdrop-plugins', '^microdrop\.')
# mpm.api.available_packages()

In [42]:
import pandas as pd

In [14]:
ws.column_dimensions[1]

TypeError: expected <type 'basestring'>

In [56]:
dt.datetime.now().date()

datetime.date(2017, 8, 19)

In [69]:
(ox.worksheet.datavalidation
 .DataValidation())

<openpyxl.worksheet.datavalidation.DataValidation object>
Parameters:
showErrorMessage=True, operator=None, prompt='Date', imeMode=None, errorStyle=None, showDropDown=None, showInputMessage=True, type='date', allowBlank=False, promptTitle='Please enter date of test', error='Please enter date in format: YYYY-MM-DD', errorTitle='Invalid date format', sqref='', formula1=None, formula2=None

In [96]:
template_path = ph.path(r'C:\Users\chris\Documents\GitHub\plugins\microdrop.mr-box-plugin\DRC Data Collection .xlsm')
df_template = pd.read_excel(template_path, sheetname='Field values')
df_template.head(3)

,Box ID,Operator,Device Type,Analyte,Standards,Sample ID Label,Sample ID,Sample Type,Accepted?,Device Problems,Device ID Glass,Device ID Printed,Date,Location,Multiplexed
0,MR Box DRC 1,Julian,Glass,Rubella IgG,None,Negative Control Measure,DRC-I-001,Blood,Yes,Power Outage,G1,P1,2017-08-20,Kinshasa,2 RV IgG + 2 RV IgM
1,MR Box DRC 2,Alex,Printed,Rubella IgM,1 (Negative),Sample Measure 1,DRC-I-002,Standard,No,Device Failed,G2,P2,2017-08-21,??,2 MV IgG + 2 MV IgM
2,MR Box DRC 3,Ryan,Roll Coated,Measles IgG,1 (Positive),Sample Measure 2,DRC-I-003,Serum,NaN,Broken Electrode,G3,P3,2017-08-22,???,2 RV IgG + 2 MV IgG


In [101]:
'"{}"'.format(','.join(df_field_values['Location'].dropna().astype(str)))

'"Kinshasa,??,???,?"'

In [142]:
cell_i = cell_map['Device ID']['value']

In [144]:
cell_i.coordinate

'D5'

In [182]:
validator_i.showErrorMessage

<openpyxl.worksheet.datavalidation.DataValidation object>
Parameters:
showErrorMessage=True, operator=None, prompt='Please select Multiplexed', imeMode=None, errorStyle=None, showDropDown=None, showInputMessage=True, type='list', allowBlank=False, promptTitle=None, error=None, errorTitle=None, sqref='D6', formula1=u'"2 RV IgG + 2 RV IgM,2 MV IgG + 2 MV IgM,2 RV IgG + 2 MV IgG,2 RV IgM + 2 MV IgM"', formula2=None

In [183]:
import datetime as dt
import types

import path_helpers as ph
import openpyxl as ox
import openpyxl.worksheet.datavalidation

# Path to the template file containing entry choices for fields.
template_path = ph.path(r'C:\Users\chris\Documents\GitHub\plugins\microdrop.mr-box-plugin\DRC Data Collection.xlsm')
# Read field values from Excel file.
df_field_values = pd.read_excel(input_path, sheetname='Field values')

# Create the workbook and worksheet we'll be working with
wb = ox.Workbook()
ws = wb.active

rows = [['Date'],
        ['Location'],
        ['Box ID'],
        ['Operator'],
        ['Device Type', None, 'Device ID'],
        ['Analyte', None, 'Multiplexed'],
        ['Standards']]

samples_headers = ['Step ID Label', 'Sample ID', 'Sample Type', 'Accepted?',
                   'What Went Wrong? (Only if not accepted)', 'Notes']

date = ox.styles.NamedStyle(name='date', number_format='yyyy-mmm-dd')

styles = {'Date': date}

# validators = {}
validators = {'Date': dict(type='date',
                           prompt='Please enter date of test (YYYY-MM-DD)',
                           error='Please enter date in format: YYYY-MM-DD',
                           errorTitle='Invalid date format',
                           allow_blank=False),
              'Device ID':
              dict(type='custom',
                   formula1=u'NOT(ISBLANK({cell}))',
                   prompt='Please enter device ID ',
                   error='Please enter device ID\n'
                   'Note: Cannot be empty',
                   errorTitle='Invalid Device ID')
}

header_font = ox.styles.Font(bold=True)
thin_border = ox.styles.Side(border_style='thin')
header_border = ox.styles.Border(*(4 * [thin_border]))

for i in (2, 4):
    ws.column_dimensions[ox.utils.get_column_letter(i)].width = 18

cell_map = {}

for i, record_i in enumerate(rows):
    for j, value_ij in enumerate(record_i):
        row_i = i + 1
        column_j = j + 1
        column_letter_j = ox.utils.get_column_letter(column_j)
        
        if value_ij is None:
            continue
        cell_ij = ws.cell(row=row_i, column=column_j, value=value_ij)
        cell_ij.font = header_font
        cell_ij.border = header_border
        ws.column_dimensions[column_letter_j].width =\
            max(len(value_ij), ws.column_dimensions[column_letter_j].width)
        cell_map[value_ij] = {'label': cell_ij,
                              'value': ws.cell(row=row_i,
                                               column=column_j + 1)}
        
samples_header_row = i + 2

for j, value_ij in enumerate(samples_headers):
    row_i = samples_header_row + 1
    column_j = j + 1
    column_letter_j = ox.utils.get_column_letter(column_j)

    if value_ij is None:
        continue
    cell_ij = ws.cell(row=row_i, column=column_j, value=value_ij)
    cell_ij.font = header_font
    cell_ij.border = header_border
    ws.column_dimensions[column_letter_j].width =\
        max(len(value_ij), ws.column_dimensions[column_letter_j].width)
    
        
cell_map['Date']['value'].value = dt.datetime.utcnow().date()

for name_i, validator_i in validators.iteritems():
    if isinstance(validator_i, dict) and 'type' in validator_i:
        cell_i = cell_map[name_i]['value']
        kwargs_i = {k: v.format(cell=cell_i.coordinate) if isinstance(v, types.StringTypes) else v
                    for k, v in validator_i.iteritems()}
        validator_i = ox.worksheet.datavalidation.DataValidation(**kwargs_i)
    ws.add_data_validation(validator_i)
    cell_i = cell_map[name_i]['value']
    validator_i.add(cell_i)

for name_i in df_field_values:
    if name_i in cell_map and (name_i not in validators or
                               ('type' not in validators[name_i] and
                               'formula1' not in validators[name_i])):
        base_validator_i =  dict(prompt='Please select {name}'
                                 .format(name=name_i),
                                 showErrorMessage=False,
#                                  error='Please select a {name} from the '
#                                  'dropdown'.format(name=name_i),
#                                  errorTitle='Invalid {name}'
#                                  .format(name=name_i),
                                 allow_blank=False)
        validator_i = base_validator_i.copy()
        validator_i.update(validators.get(name_i, {}))
        
        choices_i = df_field_values[name_i].dropna().astype(str).tolist()
        
        if validator_i.pop('allow_other', False):
            validator_i['error'] += (' or select "Other..." and enter '
                                     '{} in cell to the right'
                                     .format(name_i))
            choices_i += ['Other...']
        validator_i['type'] = 'list'
        validator_i['formula1'] = '"{}"'.format(','.join(choices_i))
        validator_i = (ox.worksheet.datavalidation
                       .DataValidation(**validator_i))
        ws.add_data_validation(validator_i)
        cell_i = cell_map[name_i]['value']
        validator_i.add(cell_i)
        print name_i, validator_i
            
for name_i, cells_i in cell_map.iteritems():
    if name_i in styles:
        cells_i['value'].style = styles[name_i]


# # Create a data-validation object with list validation
# dv = ox.worksheet.datavalidation.DataValidation(type="list",
#                                                 formula1='"Dog,Cat,Bat"',
#                                                 allow_blank=True)

# # Optionally set a custom error message
# dv.error ='Your entry is not in the list'
# dv.errorTitle = 'Invalid Entry'

# # Optionally set a custom prompt message
# dv.prompt = 'Please select from the list'
# dv.promptTitle = 'List Selection'

# # Add the data-validation object to the worksheet
# ws.add_data_validation(dv)

# # Create some cells, and add them to the data-validation object
# c1 = ws["A1"]
# c1.value = "Dog"
# dv.add(c1)
# c2 = ws["A2"]
# c2.value = "An invalid value"
# dv.add(c2)

# # Or, apply the validation to a range of cells
# dv.ranges.append('B1:B1048576')

# # Write the sheet out.  If you now open the sheet in Excel, you'll find that
# # the cells have data-validation applied.

test_path = ph.path('test.xlsx')
wb.save(test_path)
test_path.launch()

cell_map['Analyte']['value']

Box ID <openpyxl.worksheet.datavalidation.DataValidation object>
Parameters:
showErrorMessage=False, operator=None, prompt='Please select Box ID', imeMode=None, errorStyle=None, showDropDown=None, showInputMessage=True, type='list', allowBlank=False, promptTitle=None, error=None, errorTitle=None, sqref='B3', formula1=u'"MR Box DRC 1,MR Box DRC 2,MR Box DRC 3,MR Box DRC 4"', formula2=None
Operator <openpyxl.worksheet.datavalidation.DataValidation object>
Parameters:
showErrorMessage=False, operator=None, prompt='Please select Operator', imeMode=None, errorStyle=None, showDropDown=None, showInputMessage=True, type='list', allowBlank=False, promptTitle=None, error=None, errorTitle=None, sqref='B4', formula1=u'"Julian,Alex,Ryan,Richard,Tanya,Chris,Guest"', formula2=None
Device Type <openpyxl.worksheet.datavalidation.DataValidation object>
Parameters:
showErrorMessage=False, operator=None, prompt='Please select Device Type', imeMode=None, errorStyle=None, showDropDown=None, showInputMessage

<Cell Sheet.B6>

In [39]:
template_workbook.sheetnames

[]

In [31]:
import pandas as pd

In [46]:
# Read field values from Excel file.
df_field_values = pd.read_excel(input_path, sheetname='Field values')
df_field_values.head(3)

,Box ID,Operator,Device Type,Analyte,Standards,Sample ID Label,Sample ID,Sample Type,Accepted?,Device Problems,Device ID Glass,Device ID Printed,Date,Location,Multiplexed
0,MR Box DRC 1,Julian,Glass,Rubella IgG,None,Negative Control Measure,DRC-I-001,Blood,Yes,Power Outage,G1,P1,2017-08-20,Kinshasa,2 RV IgG + 2 RV IgM
1,MR Box DRC 2,Alex,Printed,Rubella IgM,1 (Negative),Sample Measure 1,DRC-I-002,Standard,No,Device Failed,G2,P2,2017-08-21,??,2 MV IgG + 2 MV IgM
2,MR Box DRC 3,Ryan,Roll Coated,Measles IgG,1 (Positive),Sample Measure 2,DRC-I-003,Serum,NaN,Broken Electrode,G3,P3,2017-08-22,???,2 RV IgG + 2 MV IgG


In [44]:
df = pd.read_excel(r'C:/Users/chris/Documents/GitHub/plugins/microdrop.mr-box-plugin/DRC Data Collection .xlsm')

In [47]:
import xlsxwriter
import path_helpers as ph

In [29]:
# workbook = xlsxwriter.Workbook(input_path)

In [74]:
import openpyxl as ox

input_book = ox.load_workbook(input_path)
output_path = input_path.parent.joinpath('{}-output.xlsx'
                                         .format(input_path.namebase))
input_path.copy(output_path)
output_book = ox.load_workbook(output_path)

In [75]:
output_book.save(output_path)

In [76]:
output_path.launch()

In [73]:
input_path.launch()

In [58]:
sheet_i = book.worksheets[0]

In [72]:
output_book.copy_worksheet(input_book.worksheets[0])

ValueError: Cannot copy between worksheets from different workbooks

In [25]:
import json


major_version = 1
# install_response = json.loads(ch.conda_exec('install', '--dry-run', '--json',
#                                             'microdrop <{}, >={}'
#                                             .format(major_version + 1,
#                                                     major_version),
#                                             verbose=False))
install_response = json.loads(ch.conda_exec('install', '--dry-run', '--json',
                                            'numpy <1.11.3',
                                            verbose=False))
print ch.format_install_info(*ch.install_info(install_response))

DEBUG:conda_helpers:Executing command: `call C:\Users\chris\Miniconda2\envs\dropbot.py\Scripts\activate.bat C:\Users\chris\Miniconda2\envs\dropbot.py & conda install --dry-run --json "numpy <1.11.3"`


KeyboardInterrupt: 

In [37]:
import re


cre_json_progress = re.compile(r'{"maxval":[^,]+,\s+"finished":[^,]+,\s+"fetch":\s+[^,]+,\s+"progress":[^}]+}')
                               
val = r'{"maxval": 133256, "finished": false, "fetch": "microdrop-laun", "progress": 0}'
cre_json_progress.search(val)

In [27]:
# ch.package_version('microdrop-launcher')

json.loads(r'''
{
  "actions": {
    "LINK": [
      "sci-bots::microdrop-launcher-0.7.2-0"
    ],
    "PREFIX": "C:\\Users\\chris\\Miniconda2\\envs\\dropbot.py",
    "SYMLINK_CONDA": [
      "C:\\Users\\chris\\Miniconda2"
    ],
    "UNLINK": [
      "wheeler-microfluidics::microdrop-launcher-0.4-0"
    ],
    "op_order": [
      "CHECK_FETCH",
      "RM_FETCHED",
      "FETCH",
      "CHECK_EXTRACT",
      "RM_EXTRACTED",
      "EXTRACT",
      "UNLINK",
      "LINK",
      "SYMLINK_CONDA"
    ]
  },
  "success": true
}
''')

{u'actions': {u'LINK': [u'sci-bots::microdrop-launcher-0.7.2-0'],
  u'PREFIX': u'C:\\Users\\chris\\Miniconda2\\envs\\dropbot.py',
  u'SYMLINK_CONDA': [u'C:\\Users\\chris\\Miniconda2'],
  u'UNLINK': [u'wheeler-microfluidics::microdrop-launcher-0.4-0'],
  u'op_order': [u'CHECK_FETCH',
   u'RM_FETCHED',
   u'FETCH',
   u'CHECK_EXTRACT',
   u'RM_EXTRACTED',
   u'EXTRACT',
   u'UNLINK',
   u'LINK',
   u'SYMLINK_CONDA']},
 u'success': True}

In [88]:
plugin_name = 'microdrop.step-label-plugin'
install_response =  mpm.api.install(plugin_name,
                                    '--no-update-dependencies', '--dry-run',
                                    channels=['microdrop-plugins'])
dry_unlinked, dry_linked = ch.install_info(install_response, split_version=True)
print dry_unlinked
print dry_linked

DEBUG:conda_helpers:Executing command: `call C:\Users\chris\Miniconda2\envs\dropbot.py\Scripts\activate.bat C:\Users\chris\Miniconda2\envs\dropbot.py & conda install -y --json -c microdrop-plugins --no-update-dependencies --dry-run microdrop.step-label-plugin`



[]
[['microdrop.step-label-plugin', '2.1', u'microdrop-plugins']]


In [105]:
import yaml


def enabled_plugins():
    enabled_plugin_packages = []
    for plugin_dir_i in mpm.api.MICRODROP_CONDA_PLUGINS.joinpath('enabled').dirs():
        properties_path_i = plugin_dir_i.joinpath('properties.yml')
        if properties_path_i.isfile():
            try:
                with properties_path_i.open('r') as input_:
                    properties_i = yaml.load(input_.read())
                if 'package_name' in properties_i:
                    properties_i['path'] = properties_path_i
                    enabled_plugin_packages.append(properties_i)
                else:
                    print properties_i
            except Exception, exception:
                print exception
                continue
    return enabled_plugin_packages

In [132]:
import types
from IPython.display import display

enabled_plugins_ = enabled_plugins() * 3
package_names = [p_i['package_name'] for p_i in enabled_plugins_]
package_names = ['foobar'] + package_names[:1] + ['microdropa']

try:
    ch.package_version(package_names, verbose=False)
except ch.PackageNotFound, exception:
    print exception
    display(exception.available)

DEBUG:conda_helpers:Executing command: `call C:\Users\chris\Miniconda2\envs\dropbot.py\Scripts\activate.bat C:\Users\chris\Miniconda2\envs\dropbot.py & conda list --json "^(foobar|microdrop.mr-box-plugin|microdropa) ?$"`



The following package(s) could not be found: `foobar`, `microdropa`


[{u'base_url': None,
  u'build_number': 0,
  u'build_string': u'0',
  u'channel': u'microdrop-plugins',
  u'dist_name': u'microdrop.mr-box-plugin-0.14-0',
  u'name': u'microdrop.mr-box-plugin',
  u'platform': None,
  u'version': u'0.14',
  u'with_features_depends': None}]

In [15]:
mpm.api.enabled_plugins(installed_only=False)

[{'package_name': 'microdrop.mr-box-plugin',
  'path': path(u'C:\\Users\\chris\\Miniconda2\\envs\\dropbot.py\\etc\\microdrop\\plugins\\enabled\\mr_box_plugin'),
  'plugin_name': 'mr_box_plugin',
  'version': '0.14',
  'versioneer': {'date': '2017-08-13T16:21:36-0400',
   'dirty': False,
   'error': None,
   'full-revisionid': '4b5e7641857fdacdb3783119d0e25f8778289131',
   'version': '0.14'}}]

In [85]:
[(package_i.split('==') if '==' in package_i
  else ['-'.join(package_i.split('-')[:-2]), package_i.split('-')[-2]])
 + [channel_i]
 for package_i, channel_i in dry_linked]
# dry_linked

[['microdrop.step-label-plugin', '2.1', u'microdrop-plugins']]

In [86]:
map(len, [range(5), range(10)])

[5, 10]

In [53]:
mpm.ui.gtk.update_plugin_dialog('microdrop.step-label-plugin', update_args=['--dry-run'])

UnboundLocalError: local variable 'invalaid_plugin_packages' referenced before assignment

In [129]:
# # XXX Potential MicroDrop update procedure #
# 
#  1. Execute `mpm.api.update` with `--dry-run` argument.
#  2. If packages are not up-to-date, notify user of proposed update
#     action(s)
#  3. If user agrees to proceed with upgrade, execute `mpm.api.update`
#     **without** the `--dry-run` argument.

package_names = ['microdrop.step-label-plugin', 'microdrop.mr-box-plugin',
                 'microdrop.droplet-planning-plugin']
update_response = mpm.api.update('--dry-run', package_name=package_names)

ch.update_info(update_response)
# if not 'actions' in update_response:
#     print 'All plugins are up to date.'
#     all_installed_versions = json.loads(ch.conda_exec('list', '--json', verbose=False))
#     installed_plugin_versions = {version_i['name']: version_i
#                                  for version_i in all_installed_versions
#                                  if version_i['name'] in package_names}
#     for package_name_i in package_names:
#         version_info_i = installed_plugin_versions[package_name_i]
#         print '{name}: {version} ({channel})'.format(**version_info_i)
# else:
#     print update_response

(False,
 [['microdrop.droplet-planning-plugin==2.1.post9', u'microdrop-plugins'],
  ['microdrop.mr-box-plugin==0.9', u'microdrop-plugins']],
 [['conda-helpers==0.6.1.0', u'sci-bots'],
  ['microdrop-plugin-manager==0.19', u'sci-bots'],
  ['microdrop.droplet-planning-plugin==2.2.1', u'microdrop-plugins'],
  ['microdrop.mr-box-plugin==0.14', u'microdrop-plugins']])

In [125]:
update_response

{u'actions': [{u'LINK': [{u'base_url': None,
     u'build_number': 0,
     u'build_string': u'py27_0',
     u'channel': u'sci-bots',
     u'dist_name': u'conda-helpers-0.6.1.0-py27_0',
     u'name': u'conda-helpers',
     u'platform': None,
     u'version': u'0.6.1.0',
     u'with_features_depends': None},
    {u'base_url': None,
     u'build_number': 0,
     u'build_string': u'0',
     u'channel': u'sci-bots',
     u'dist_name': u'microdrop-plugin-manager-0.19-0',
     u'name': u'microdrop-plugin-manager',
     u'platform': None,
     u'version': u'0.19',
     u'with_features_depends': None}],
   u'PREFIX': u'C:\\Users\\chris\\Miniconda2\\envs\\dropbot.py',
   u'SYMLINK_CONDA': [u'C:\\Users\\chris\\Miniconda2'],
   u'op_order': [u'CHECK_FETCH',
    u'RM_FETCHED',
    u'FETCH',
    u'CHECK_EXTRACT',
    u'RM_EXTRACTED',
    u'EXTRACT',
    u'UNLINK',
    u'LINK',
    u'SYMLINK_CONDA']}],
 u'dry_run': True,
 u'success': True}

In [82]:
print update_response

{u'success': True, u'dry_run': True, u'actions': [{u'SYMLINK_CONDA': [u'C:\\Users\\chris\\Miniconda2'], u'op_order': [u'CHECK_FETCH', u'RM_FETCHED', u'FETCH', u'CHECK_EXTRACT', u'RM_EXTRACTED', u'EXTRACT', u'UNLINK', u'LINK', u'SYMLINK_CONDA'], u'PREFIX': u'C:\\Users\\chris\\Miniconda2\\envs\\dropbot.py', u'UNLINK': [{u'dist_name': u'microdrop.droplet-planning-plugin-2.1.post9-0', u'build_number': 0, u'name': u'microdrop.droplet-planning-plugin', u'build_string': u'0', u'base_url': None, u'with_features_depends': None, u'platform': None, u'version': u'2.1.post9', u'channel': u'microdrop-plugins'}, {u'dist_name': u'microdrop.mr-box-plugin-0.9-0', u'build_number': 0, u'name': u'microdrop.mr-box-plugin', u'build_string': u'0', u'base_url': None, u'with_features_depends': None, u'platform': None, u'version': u'0.9', u'channel': u'microdrop-plugins'}], u'LINK': [{u'dist_name': u'microdrop.droplet-planning-plugin-2.2.1-0', u'build_number': 0, u'name': u'microdrop.droplet-planning-plugin', u'

In [25]:
update_response__uptodate = update_response

In [22]:
update_response__updated = update_response

In [79]:
update_response = update_response__updated
update_response

{u'actions': {u'LINK': [u'wheeler-microfluidics::mr-box-peripheral-board-0.22.0-py27_0',
   u'microdrop-plugins::microdrop.droplet-planning-plugin-2.2.1-0',
   u'microdrop-plugins::microdrop.mr-box-plugin-0.14-0'],
  u'PREFIX': u'C:\\Users\\chris\\Miniconda2\\envs\\dropbot.py',
  u'SYMLINK_CONDA': [u'C:\\Users\\chris\\Miniconda2'],
  u'UNLINK': [u'microdrop-plugins::microdrop.droplet-planning-plugin-2.1.post9-0',
   u'microdrop-plugins::microdrop.mr-box-plugin-0.9-0',
   u'wheeler-microfluidics::mr-box-peripheral-board-0.20.0-py27_0'],
  u'op_order': [u'CHECK_FETCH',
   u'RM_FETCHED',
   u'FETCH',
   u'CHECK_EXTRACT',
   u'RM_EXTRACTED',
   u'EXTRACT',
   u'UNLINK',
   u'LINK',
   u'SYMLINK_CONDA']},
 u'success': True}

In [50]:
update_response['actions']

{u'LINK': [u'microdrop-plugins::microdrop.droplet-planning-plugin-2.2.1-0',
  u'microdrop-plugins::microdrop.mr-box-plugin-0.14-0'],
 u'PREFIX': u'C:\\Users\\chris\\Miniconda2\\envs\\dropbot.py',
 u'SYMLINK_CONDA': [u'C:\\Users\\chris\\Miniconda2'],
 u'UNLINK': [u'microdrop-plugins::microdrop.droplet-planning-plugin-2.1.post9-0',
  u'microdrop-plugins::microdrop.mr-box-plugin-0.9-0'],
 u'op_order': [u'CHECK_FETCH',
  u'RM_FETCHED',
  u'FETCH',
  u'CHECK_EXTRACT',
  u'RM_EXTRACTED',
  u'EXTRACT',
  u'UNLINK',
  u'LINK',
  u'SYMLINK_CONDA']}

In [103]:
dry_run = True
if dry_run:
    unlink_packages = [['{}=={}'.format(v['name'], v['version']), v['channel']] for v in update_response['actions'][0]['UNLINK']]
    link_packages = [['{}=={}'.format(v['name'], v['version']), v['channel']] for v in update_response['actions'][0]['LINK']]
else:
    unlink_packages = [v.split('::')[::-1]
                       for v in update_response['actions']['UNLINK']]
    link_packages = [v.split('::')[::-1]
                     for v in update_response['actions']['LINK']]

sorted_from = sorted(unlink_packages)
sorted_to = sorted(link_packages)
if dry_run:
    print 'Update from:'
else:
    print 'Updated from:'
for package_i, channel_i in sorted_from:
    print ' - `{}` (from `{}`)'.format(package_i, channel_i)
print 'to:'
for package_i, channel_i in sorted_to:
    print ' - `{}` (from `{}`)'.format(package_i, channel_i)

Update from:
 - `microdrop.droplet-planning-plugin==2.1.post9` (from `microdrop-plugins`)
 - `microdrop.mr-box-plugin==0.9` (from `microdrop-plugins`)
to:
 - `microdrop.droplet-planning-plugin==2.2.1` (from `microdrop-plugins`)
 - `microdrop.mr-box-plugin==0.14` (from `microdrop-plugins`)


In [17]:
import json

import conda_helpers as ch

In [20]:
all_installed_versions = json.loads(ch.conda_exec('list', '--json', verbose=False))
installed_plugin_versions = {version_i['name']: version_i
                             for version_i in all_installed_versions
                             if version_i['name'] in package_names}
installed_plugin_versions

{u'microdrop.droplet-planning-plugin': {u'base_url': None,
  u'build_number': 0,
  u'build_string': u'0',
  u'channel': u'microdrop-plugins',
  u'dist_name': u'microdrop.droplet-planning-plugin-2.2.1-0',
  u'name': u'microdrop.droplet-planning-plugin',
  u'platform': None,
  u'version': u'2.2.1',
  u'with_features_depends': None},
 u'microdrop.mr-box-plugin': {u'base_url': None,
  u'build_number': 0,
  u'build_string': u'0',
  u'channel': u'microdrop-plugins',
  u'dist_name': u'microdrop.mr-box-plugin-0.14-0',
  u'name': u'microdrop.mr-box-plugin',
  u'platform': None,
  u'version': u'0.14',
  u'with_features_depends': None},
 u'microdrop.step-label-plugin': {u'base_url': None,
  u'build_number': 0,
  u'build_string': u'0',
  u'channel': u'microdrop-plugins',
  u'dist_name': u'microdrop.step-label-plugin-2.1-0',
  u'name': u'microdrop.step-label-plugin',
  u'platform': None,
  u'version': u'2.1',
  u'with_features_depends': None}}